In [23]:
import pandas as pd

data1 = pd.read_csv("IDCJAC0009_070014_1800_Data.csv", na_values=[""])
data2 = pd.read_csv("IDCJAC0010_070014_1800_Data.csv", na_values=[""])
data3 = pd.read_csv("IDCJAC0009_070351_1800_Data.csv", na_values=[""])
data4 = pd.read_csv("IDCJAC0010_070351_1800_Data.csv", na_values=[""])

climate_data = pd.DataFrame(columns={"rainfall", "max_temp"})

def isNaN(x):
    return x != x

for i, row in data1.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
    climate_data.loc[date, "rainfall"] = row["Rainfall amount (millimetres)"]

for i, row in data2.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
    climate_data.loc[date, "max_temp"] = row["Maximum temperature (Degree C)"]

for i, row in data3.iterrows():
    if not isNaN(row["Rainfall amount (millimetres)"]):
        date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
        climate_data.loc[date, "rainfall"] = row["Rainfall amount (millimetres)"]
        
for i, row in data4.iterrows():
    if not isNaN(row["Maximum temperature (Degree C)"]):
        date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
        climate_data.loc[date, "max_temp"] = row["Maximum temperature (Degree C)"]
        
climate_data

,max_temp,rainfall
1939-01-01,NaN,NaN
1939-01-02,NaN,NaN
1939-01-03,NaN,NaN
1939-01-04,NaN,NaN
1939-01-05,NaN,NaN
...,...,...
2015-08-12,7.5,NaN
2015-10-26,27,NaN
2017-03-09,23.5,NaN
2017-04-01,22.3,NaN


In [25]:
climate_data.to_csv("climate_data.csv", index_label="date")